In [2]:
!pip install pymystem3


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import pandas as pd
import numpy as np
import string
import nltk

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

from nltk.stem.snowball import SnowballStemmer 
from tqdm.auto import tqdm, trange
from nltk.stem import *
from nltk.corpus import stopwords
# from pymystem3 import Mystem
from string import punctuation
from nltk import word_tokenize

# nltk.download('stopwords')
# nltk.download('punkt')

In [5]:
from google.colab import files
uploaded = files.upload()

Saving rls_.csv to rls_.csv


In [4]:
df = pd.read_csv('rls_.csv', sep=';', encoding='cp1251')

In [5]:
df.shape

(10695, 6)

In [6]:
df.head(5)

,ИД правила,Класс,Левое ДВ,Правое ДВ,Правило взаимодействия,Исходники
0,38039,2,4-нитро-N-[(1RS)-1-(4-фторфенил)-2-(1-этилпипе...,Верапамил*,Из-за риска развития брадикардии и нарушений п...,из-за риска развития брадикардии и нарушений ...
1,38041,2,4-нитро-N-[(1RS)-1-(4-фторфенил)-2-(1-этилпипе...,Дигоксин*,Из-за риска развития брадикардии и нарушений п...,из-за риска развития брадикардии и нарушений ...
2,38040,2,4-нитро-N-[(1RS)-1-(4-фторфенил)-2-(1-этилпипе...,Дилтиазем*,Из-за риска развития брадикардии и нарушений п...,из-за риска развития брадикардии и нарушений ...
3,38006,3,4-нитро-N-[(1RS)-1-(4-фторфенил)-2-(1-этилпипе...,Амиодарон*,Противопоказано сочетание 4-нитро-N-[(1RS)-1-(...,противопоказано сочетание препарата Рефралон®...
4,38009,3,4-нитро-N-[(1RS)-1-(4-фторфенил)-2-(1-этилпипе...,Бретилия тозилат*,Противопоказано сочетание 4-нитро-N-[(1RS)-1-(...,противопоказано сочетание препарата Рефралон®...


In [7]:
df['Класс'].value_counts()

2    8463
3    1221
1    1011
Name: Класс, dtype: int64

In [6]:
def remove_punctuation(text):
    return "".join([ch if ch not in string.punctuation else ' ' for ch in text])

def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text])

import re
def remove_multiple_spaces(text):
	return re.sub(r'\s+', ' ', text, flags=re.I)

mystem = Mystem() 

russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['…', '«', '»', '...'])
def lemmatize_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords and token != " "]
    text = " ".join(tokens)
    return text

Installing mystem to C:\Users\Анечка/.local/bin\mystem.exe from http://download.cdn.yandex.net/mystem/mystem-3.1-win-64bit.zip


In [7]:
preproccessing = lambda text: (remove_multiple_spaces(remove_numbers(remove_punctuation(text))))
df['preproccessed'] = list(map(preproccessing, df['Исходники']))
prep_text = [remove_multiple_spaces(remove_numbers(remove_punctuation(text.lower()))) for text in tqdm(df['Исходники'])]

100%|██████████| 10695/10695 [00:01<00:00, 7473.78it/s]


In [10]:
len(prep_text)


10695

In [11]:
type(prep_text)

list

In [8]:
df['prep_text']=prep_text

In [9]:
df

,ИД правила,Класс,Левое ДВ,Правое ДВ,Правило взаимодействия,Исходники,preproccessed,prep_text
0,38039,2,4-нитро-N-[(1RS)-1-(4-фторфенил)-2-(1-этилпипе...,Верапамил*,Из-за риска развития брадикардии и нарушений п...,из-за риска развития брадикардии и нарушений ...,из за риска развития брадикардии и нарушений ...,из за риска развития брадикардии и нарушений ...
1,38041,2,4-нитро-N-[(1RS)-1-(4-фторфенил)-2-(1-этилпипе...,Дигоксин*,Из-за риска развития брадикардии и нарушений п...,из-за риска развития брадикардии и нарушений ...,из за риска развития брадикардии и нарушений ...,из за риска развития брадикардии и нарушений ...
2,38040,2,4-нитро-N-[(1RS)-1-(4-фторфенил)-2-(1-этилпипе...,Дилтиазем*,Из-за риска развития брадикардии и нарушений п...,из-за риска развития брадикардии и нарушений ...,из за риска развития брадикардии и нарушений ...,из за риска развития брадикардии и нарушений ...
3,38006,3,4-нитро-N-[(1RS)-1-(4-фторфенил)-2-(1-этилпипе...,Амиодарон*,Противопоказано сочетание 4-нитро-N-[(1RS)-1-(...,противопоказано сочетание препарата Рефралон®...,противопоказано сочетание препарата Рефралон®...,противопоказано сочетание препарата рефралон®...
4,38009,3,4-нитро-N-[(1RS)-1-(4-фторфенил)-2-(1-этилпипе...,Бретилия тозилат*,Противопоказано сочетание 4-нитро-N-[(1RS)-1-(...,противопоказано сочетание препарата Рефралон®...,противопоказано сочетание препарата Рефралон®...,противопоказано сочетание препарата рефралон®...
...,...,...,...,...,...,...,...,...
10690,24877,2,Дроспиренон + Этинилэстрадиол + [Кальция левом...,Омепразол*,На основании исследований взаимодействия in vi...,"Ярина плюс, табл. п.п.о., 23.01.2018. На основ...",Ярина плюс табл п п о На основании исследовани...,ярина плюс табл п п о на основании исследовани...
10691,24878,2,Дроспиренон + Этинилэстрадиол + [Кальция левом...,Симвастатин*,На основании исследований взаимодействия in vi...,"Ярина плюс, табл. п.п.о., 23.01.2018. На основ...",Ярина плюс табл п п о На основании исследовани...,ярина плюс табл п п о на основании исследовани...
10692,24880,2,Дроспиренон + Этинилэстрадиол + [Кальция левом...,Пириметамин*,Кальция левомефолат (в составе комбинации дрос...,"Ярина плюс, табл. п.п.о., 23.01.2018. Фолаты м...",Ярина плюс табл п п о Фолаты могут изменять фа...,ярина плюс табл п п о фолаты могут изменять фа...
10693,59753,3,Дроспиренон + Этинилэстрадиол + [Кальция левом...,Дасабувир; Омбитасвир + Паритапревир + Ритонавир,Противопоказано совместное применение комбинац...,"Ярина плюс, табл. п.п.о., 23.01.2018.\nПротиво...",Ярина плюс табл п п о Противопоказания Совмест...,ярина плюс табл п п о противопоказания совмест...


In [10]:
stemmer = SnowballStemmer("russian") 
russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['…', '«', '»', '...', 'т.д.', 'т', 'д'])
text = df['prep_text'][0]
word_tokenize(text) 

['из',
 'за',
 'риска',
 'развития',
 'брадикардии',
 'и',
 'нарушений',
 'проводимости',
 'не',
 'рекомендуется',
 'введение',
 'препарата',
 'на',
 'фоне',
 'приема',
 'бета',
 'адреноблокаторов',
 'блокаторов',
 '«',
 'медленных',
 '»',
 'кальциевых',
 'каналов',
 'урежающих',
 'чсс',
 'верапамил',
 'дилтиазем',
 'дигоксина']

In [11]:
stemmed_texts_list = []
for text in tqdm(df['prep_text']):
    tokens = word_tokenize(text)    
    stemmed_tokens = [stemmer.stem(token) for token in tokens if token not in russian_stopwords]
    text = " ".join(stemmed_tokens)
    stemmed_texts_list.append(text)

df['text_stem'] = stemmed_texts_list

100%|██████████| 10695/10695 [00:25<00:00, 426.06it/s]


In [12]:
def remove_stop_words(text):
    tokens = word_tokenize(text) 
    tokens = [token for token in tokens if token not in russian_stopwords and token != ' ']
    return " ".join(tokens)
  
sw_texts_list = []
for text in tqdm(df['prep_text']):
    tokens = word_tokenize(text)    
    tokens = [token for token in tokens if token not in russian_stopwords and token != ' ']
    text = " ".join(tokens)
    sw_texts_list.append(text)

df['text_sw'] = sw_texts_list

100%|██████████| 10695/10695 [00:04<00:00, 2559.84it/s]


In [13]:
df['text_sw'][0]

'риска развития брадикардии нарушений проводимости рекомендуется введение препарата фоне приема бета адреноблокаторов блокаторов медленных кальциевых каналов урежающих чсс верапамил дилтиазем дигоксина'

In [14]:
df.to_csv('rls_stemmed.csv')

In [15]:
df['text_stem'][0]

'риск развит брадикард нарушен проводим рекоменд введен препарат фон прием бет адреноблокатор блокатор медлен кальциев канал урежа чсс верапам дилтиаз дигоксин'

In [16]:
lemm_texts_list = []
for text in tqdm(df['text_sw']):

    try:
        text_lem = mystem.lemmatize(text)
        tokens = [token for token in text_lem if token != ' ' and token not in russian_stopwords]
        text = " ".join(tokens)
        lemm_texts_list.append(text)
    except Exception as e:
        print(e)
    
df['text_lemm'] = lemm_texts_list

  0%|          | 40/10695 [00:47<3:32:48,  1.20s/it]


KeyboardInterrupt: 

In [21]:
def lemmatize_text(text):
    text_lem = mystem.lemmatize(text)
    tokens = [token for token in text_lem if token != ' ']
    return " ".join(tokens)

In [23]:
df.to_csv('lemm.csv')

In [3]:
df_lemm = pd.read_csv('lemm.csv', encoding = 'utf-8')

In [26]:
df_lemm.head(2)

,Unnamed: 0,ИД правила,Класс,Левое ДВ,Правое ДВ,Правило взаимодействия,Исходники,preproccessed,prep_text,text_stem,text_sw,text_lemm
0,0,38039,2,4-нитро-N-[(1RS)-1-(4-фторфенил)-2-(1-этилпипе...,Верапамил*,Из-за риска развития брадикардии и нарушений п...,из-за риска развития брадикардии и нарушений ...,из за риска развития брадикардии и нарушений ...,из за риска развития брадикардии и нарушений ...,риск развит брадикард нарушен проводим рекомен...,риска развития брадикардии нарушений проводимо...,риск развитие брадикардия нарушение проводимос...
1,1,38041,2,4-нитро-N-[(1RS)-1-(4-фторфенил)-2-(1-этилпипе...,Дигоксин*,Из-за риска развития брадикардии и нарушений п...,из-за риска развития брадикардии и нарушений ...,из за риска развития брадикардии и нарушений ...,из за риска развития брадикардии и нарушений ...,риск развит брадикард нарушен проводим рекомен...,риска развития брадикардии нарушений проводимо...,риск развитие брадикардия нарушение проводимос...


In [58]:
X = df_lemm['text_sw']
y = df_lemm['Класс']

In [59]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [60]:
my_tags = df_lemm['Класс'].unique()
my_tags

array([2, 3, 1], dtype=int64)

## BOW with Keras

In [62]:
df_lemm.head()

,Unnamed: 0,ИД правила,Класс,Левое ДВ,Правое ДВ,Правило взаимодействия,Исходники,preproccessed,prep_text,text_stem,text_sw,text_lemm
0,0,38039,2,4-нитро-N-[(1RS)-1-(4-фторфенил)-2-(1-этилпипе...,Верапамил*,Из-за риска развития брадикардии и нарушений п...,из-за риска развития брадикардии и нарушений ...,из за риска развития брадикардии и нарушений ...,из за риска развития брадикардии и нарушений ...,риск развит брадикард нарушен проводим рекомен...,риска развития брадикардии нарушений проводимо...,риск развитие брадикардия нарушение проводимос...
1,1,38041,2,4-нитро-N-[(1RS)-1-(4-фторфенил)-2-(1-этилпипе...,Дигоксин*,Из-за риска развития брадикардии и нарушений п...,из-за риска развития брадикардии и нарушений ...,из за риска развития брадикардии и нарушений ...,из за риска развития брадикардии и нарушений ...,риск развит брадикард нарушен проводим рекомен...,риска развития брадикардии нарушений проводимо...,риск развитие брадикардия нарушение проводимос...
2,2,38040,2,4-нитро-N-[(1RS)-1-(4-фторфенил)-2-(1-этилпипе...,Дилтиазем*,Из-за риска развития брадикардии и нарушений п...,из-за риска развития брадикардии и нарушений ...,из за риска развития брадикардии и нарушений ...,из за риска развития брадикардии и нарушений ...,риск развит брадикард нарушен проводим рекомен...,риска развития брадикардии нарушений проводимо...,риск развитие брадикардия нарушение проводимос...
3,3,38006,3,4-нитро-N-[(1RS)-1-(4-фторфенил)-2-(1-этилпипе...,Амиодарон*,Противопоказано сочетание 4-нитро-N-[(1RS)-1-(...,противопоказано сочетание препарата Рефралон®...,противопоказано сочетание препарата Рефралон®...,противопоказано сочетание препарата рефралон®...,противопоказа сочетан препарат рефралон® препа...,противопоказано сочетание препарата рефралон® ...,противопоказанный сочетание препарат рефралон ...
4,4,38009,3,4-нитро-N-[(1RS)-1-(4-фторфенил)-2-(1-этилпипе...,Бретилия тозилат*,Противопоказано сочетание 4-нитро-N-[(1RS)-1-(...,противопоказано сочетание препарата Рефралон®...,противопоказано сочетание препарата Рефралон®...,противопоказано сочетание препарата рефралон®...,противопоказа сочетан препарат рефралон® препа...,противопоказано сочетание препарата рефралон® ...,противопоказанный сочетание препарат рефралон ...


In [129]:
# Model training
import itertools
import os
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils


df_lemm = pd.read_csv('lemm.csv', encoding='utf-8')

df_lemm_test = df_lemm[:2000]
# df_lemm = df_lemm[2001:]

X = df_lemm['text_sw']
y = df_lemm['Класс']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Naive Bayes for help: 
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

with open('bow_model_short/addition_model.pickle', 'wb') as f:
    pickle.dump(nb, f, protocol=pickle.HIGHEST_PROTOCOL)

train_posts = X_train
train_tags = y_train

test_posts = X_test 
test_tags = y_test

tokenize = text.Tokenizer()
tokenize.fit_on_texts(train_posts) # only fit on train

x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_posts)

encoder = LabelEncoder()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

num_classes = 3
y_train = utils.to_categorical(y_train)
y_test = utils.to_categorical(y_test)

batch_size = 64
epochs = 40

# Build the model
model = Sequential()
# model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['Precision', 'Accuracy'])
              
model.fit(
    x_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_split=0.1
    )
model.save('bow_model_short')

with open('bow_model_short/tokenizer.pickle', 'wb') as f:
    pickle.dump(tokenize, f, protocol=pickle.HIGHEST_PROTOCOL)

Epoch 1/40
121/121 [==============================] - 6s 48ms/step - loss: 0.7054 - precision: 0.8641 - Accuracy: 0.7944 - val_loss: 0.5363 - val_precision: 0.8592 - val_Accuracy: 0.8376
Epoch 2/40
121/121 [==============================] - 6s 47ms/step - loss: 0.4536 - precision: 0.9004 - Accuracy: 0.8678 - val_loss: 0.4130 - val_precision: 0.8943 - val_Accuracy: 0.8703
Epoch 3/40
121/121 [==============================] - 6s 48ms/step - loss: 0.3654 - precision: 0.9191 - Accuracy: 0.8961 - val_loss: 0.3469 - val_precision: 0.9144 - val_Accuracy: 0.8890
Epoch 4/40
121/121 [==============================] - 6s 50ms/step - loss: 0.3089 - precision: 0.9286 - Accuracy: 0.9100 - val_loss: 0.3072 - val_precision: 0.9201 - val_Accuracy: 0.9030
Epoch 5/40
121/121 [==============================] - 6s 46ms/step - loss: 0.2733 - precision: 0.9385 - Accuracy: 0.9234 - val_loss: 0.2817 - val_precision: 0.9197 - val_Accuracy: 0.9089
Epoch 6/40
121/121 [==============================] - 6s 47ms/ste

In [113]:
len(df_lemm)

10695

In [130]:
import pandas as pd
import numpy as np
import string
import nltk
import pickle 

from nltk.stem.snowball import SnowballStemmer 
from tqdm.auto import tqdm, trange
from nltk.stem import *
from nltk.corpus import stopwords
# from pymystem3 import Mystem
from string import punctuation
from nltk import word_tokenize

import re
import tensorflow as tf

from itertools import chain 


def remove_punctuation(text):
    return "".join([ch if ch not in string.punctuation else ' ' for ch in text])

def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text])

def remove_multiple_spaces(text):
	return re.sub(r'\s+', ' ', text, flags=re.I)

def remove_stop_words(text):
    tokens = word_tokenize(text) 
    tokens = [token for token in tokens if token not in russian_stopwords and token != ' ']
    return " ".join(tokens)

def all_preprocessing(df: pd.DataFrame):

    preproccessing = lambda text: (remove_multiple_spaces(remove_numbers(remove_punctuation(text))))
    df['preproccessed'] = list(map(preproccessing, df['Исходники']))
    prep_text = [remove_multiple_spaces(remove_numbers(remove_punctuation(text.lower()))) for text in tqdm(df['Исходники'])]

    df['prep_text'] = prep_text

    stemmer = SnowballStemmer("russian") 
    russian_stopwords = stopwords.words("russian")
    russian_stopwords.extend(['…', '«', '»', '...', 'т.д.', 'т', 'д'])
    text = df['prep_text'][0]

    stemmed_texts_list = []
    for text in tqdm(df['prep_text']):
        tokens = word_tokenize(text)    
        stemmed_tokens = [stemmer.stem(token) for token in tokens if token not in russian_stopwords]
        text = " ".join(stemmed_tokens)
        stemmed_texts_list.append(text)

    df['text_stem'] = stemmed_texts_list
    
    sw_texts_list = []
    for text in tqdm(df['prep_text']):
        tokens = word_tokenize(text)    
        tokens = [token for token in tokens if token not in russian_stopwords and token != ' ']
        text = " ".join(tokens)
        sw_texts_list.append(text)
    
    # Lemmatized column only: 
    return sw_texts_list

# df = pd.read_csv('rls_.csv', sep=';', encoding='cp1251')
df = df_lemm_test

# Load tokenizer: 
with open('bow_model_short/tokenizer.pickle', 'rb') as handle:
    tokenize = pickle.load(handle)

# Load models: 
model = tf.keras.models.load_model('bow_model_short')
with open('bow_model_short/addition_model.pickle', 'rb') as m:
    nb = pickle.load(m)

preprocessed_input = all_preprocessing(df) # list
initials_list, result_list = [], []
length = len(preprocessed_input)

for num in range(length):
    if isinstance(preprocessed_input, pd.DataFrame): 
        if not (num in preprocessed_input.index): 
            continue
        row_0 = preprocessed_input[preprocessed_input.index == num]
    elif isinstance(preprocessed_input, list): 
        row_0 = [preprocessed_input[num]]
    elif isinstance(preprocessed_input, pd.Series): 
        if not (num in preprocessed_input.index): 
            continue
        row_0 = [preprocessed_input[num]]
    row_tokenized = tokenize.texts_to_matrix(row_0)
    class_prediction = []
    count = 5
    while (len(class_prediction) == 0) & (count > 0):  
        preds_0 = list(chain(*model.predict(row_tokenized).tolist()))
        preds_f = [round(cls) for cls in preds_0]
        class_prediction = [ind+1 for (ind, obj) in enumerate(preds_f) if obj == 1]
        count -= 1
    if len(class_prediction): 
        class_prediction = class_prediction[0]
    else: 
        print('NB model prediction')
        class_prediction = nb.predict(row_0)[0]
    initial = df[df.index == num]['Класс'].values

    initials_list.append(row_0)
    result_list.append(class_prediction)


result = pd.DataFrame({
    'Исходники': initials_list, 
    'Предсказанный класс': result_list
})

# result = all_predictions(df=df[:int(len(df))])

y_pred = result['Предсказанный класс']
result.to_csv('Результат')

C:\Users\Анечка\AppData\Local\Temp\ipykernel_13396\4170304509.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['preproccessed'] = list(map(preproccessing, df['Исходники']))


  0%|          | 0/2000 [00:00<?, ?it/s]

C:\Users\Анечка\AppData\Local\Temp\ipykernel_13396\4170304509.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prep_text'] = prep_text


  0%|          | 0/2000 [00:00<?, ?it/s]

C:\Users\Анечка\AppData\Local\Temp\ipykernel_13396\4170304509.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text_stem'] = stemmed_texts_list


  0%|          | 0/2000 [00:00<?, ?it/s]

1/1 [==============================] - 0s 18ms/step
NB model prediction
1/1 [==============================] - 0s 19ms/step
NB model prediction
1/1 [==============================] - 0s 21ms/step


AttributeError: module 'pandas' has no attribute 'to_csv'

In [131]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.metrics import classification_report

# df = pd.read_csv('rls_.csv', sep=';', encoding='cp1251')
# data = df=df[:int(len(df)/4)]

y = [obj[0] for obj in result['Исходники']]
# y_pred = y_pred[list(y.index)]

# print(classification_report(y_true=y, y_pred=y_pred))
print(accuracy_score(y_true=y, y_pred=y_pred))
print(precision_score(y_true=y, y_pred=y_pred, average='weighted'))
print(recall_score(y_true=y, y_pred=y_pred, average='weighted'))
print(f1_score(y_true=y, y_pred=y_pred, average='weighted'))

0.905982905982906
0.9976091476091477
0.905982905982906
0.9495436129311235


a:\Programs\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [73]:
type(X_test)

pandas.core.series.Series

Accuracy для лем обучения и нелем входных: 0.836  
Accuracy для нелем обучения и нелем входных: 0 (??)  
Accuracy для 100 эпох НЕ класса 2 (его больше всего, его предсказывает лучше всего): 0.5 - ужасно, надо делать лемматизацию и для входных 
Acc для 3 эпох НЕ класса 2 

Acc 0.75

In [103]:
accur

0.7702702702702703

In [539]:
accur = good/(good + bad)
print(accur)

0.49


In [563]:
from itertools import chain 


num = 125
for num in range(0, 100, 5):
    row_0 = df_lemm[df_lemm.index == num]['text_lemm']
    # print('Row: ', row_0.values)
    row_tokenized = tokenize.texts_to_matrix(row_0)

    preds_0 = list(chain(*model.predict(row_tokenized).tolist()))
    preds_f = [round(cls) for cls in preds_0]
    class_prediction = [ind+1 for (ind, obj) in enumerate(preds_f) if obj == 1][0]

    print('Initial', *df_lemm[df_lemm.index == num]['Класс'].values)
    print('Predicted', class_prediction)

1/1 [==============================] - 0s 23ms/step
Initial 2
Predicted 2
1/1 [==============================] - 0s 22ms/step
Initial 3
Predicted 3
1/1 [==============================] - 0s 18ms/step
Initial 3
Predicted 3
1/1 [==============================] - 0s 21ms/step
Initial 3
Predicted 3
1/1 [==============================] - 0s 18ms/step
Initial 3
Predicted 3
1/1 [==============================] - 0s 20ms/step
Initial 3
Predicted 3
1/1 [==============================] - 0s 21ms/step
Initial 3
Predicted 3
1/1 [==============================] - 0s 22ms/step
Initial 3
Predicted 3
1/1 [==============================] - 0s 25ms/step
Initial 2
Predicted 2
1/1 [==============================] - 0s 24ms/step
Initial 2
Predicted 2
1/1 [==============================] - 0s 20ms/step
Initial 2
Predicted 2
1/1 [==============================] - 0s 20ms/step
Initial 2
Predicted 2
1/1 [==============================] - 0s 20ms/step
Initial 2
Predicted 2
1/1 [==============================] -

In [384]:
df_lemm[df_lemm.index == 10]

,Unnamed: 0,ИД правила,Класс,Левое ДВ,Правое ДВ,Правило взаимодействия,Исходники,preproccessed,prep_text,text_stem,text_sw,text_lemm
10,10,37906,3,4-нитро-N-[(1RS)-1-(4-фторфенил)-2-(1-этилпипе...,Хинидин,Противопоказано сочетание 4-нитро-N-[(1RS)-1-(...,противопоказано сочетание препарата Рефралон®...,противопоказано сочетание препарата Рефралон®...,противопоказано сочетание препарата рефралон®...,противопоказа сочетан препарат рефралон® препа...,противопоказано сочетание препарата рефралон® ...,противопоказанный сочетание препарат рефралон ...


## Наивный байесовский классификатор

In [42]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

In [43]:
%%time
nb.fit(X_train, y_train)

CPU times: total: 328 ms
Wall time: 381 ms


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [44]:
%%time
from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

CPU times: total: 141 ms
Wall time: 133 ms


In [45]:
y_pred[0]

2

In [46]:
print(X_test[0], y_test[0], y_pred[0])

риск развитие брадикардия нарушение проводимость рекомендоваться введение препарат фон прием бета адреноблокатор блокатор медленный кальциевый канал урежать чсс верапамил дилтиазем дигоксин 
 2 2


In [47]:
from sklearn.metrics import accuracy_score

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.8666251168588345
              precision    recall  f1-score   support

           1       0.91      0.40      0.56       300
           2       0.86      0.99      0.92      2515
           3       0.95      0.41      0.57       394

    accuracy                           0.87      3209
   macro avg       0.91      0.60      0.68      3209
weighted avg       0.88      0.87      0.84      3209



## Метод опорных векторов

In [65]:
sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])

In [66]:
%%time
sgd.fit(X_train, y_train)

ValueError: y should be a 1d array, got an array of shape (7486, 3) instead.

In [46]:
%%time
y_pred = sgd.predict(X_test)

Wall time: 153 ms


In [48]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.8884387659707074
              precision    recall  f1-score   support

           1       0.91      0.42      0.58       300
           2       0.88      0.99      0.93      2515
           3       0.96      0.59      0.73       394

    accuracy                           0.89      3209
   macro avg       0.92      0.67      0.75      3209
weighted avg       0.89      0.89      0.87      3209



## Логистическая регрессия

In [49]:
logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])

In [50]:
%%time
logreg.fit(X_train, y_train)

Wall time: 1.79 s


C:\Users\DarkSidius\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(C=100000.0, n_jobs=1))])

In [51]:
%%time
y_pred = logreg.predict(X_test)

Wall time: 147 ms


In [ ]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,targe))